# <div style=" text-align: center; font-weight: bold">Phase 02: Preprocessing data</div>


## **1. Import thư viện**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
# !pip install unicodedata
import unicodedata
import seaborn as sns

## **2. Khám phá dữ liệu**

### **2.1 Đọc dữ liệu**:

In [2]:
real_estate_for_sale_df = pd.read_csv('../Data/real_estate_for_sale.csv', encoding='utf-8')
real_estate_for_sale_df.head()

,Address,Type,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Đường Lê Văn Việt, Phường Long Thạnh Mỹ, Quận ...",Nhà riêng,66 m²,Thỏa thuận,4 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,03/05/2025,13/05/2025,Tin thường,42376273
1,"Dự án Fuji Residence, Đường Đỗ Xuân Hợp, Phườn...",Căn hộ chung cư,55 m²,"2,7 tỷ",1 phòng,NaN,NaN,Đầy đủ,Tây - Bắc,Sổ đỏ/ Sổ hồng,03/05/2025,13/05/2025,Tin thường,41283793
2,"Đường Lê Văn Sỹ, Phường 1, Tân Bình, Hồ Chí Minh",Nhà riêng,45 m²,8 tỷ,4 phòng,NaN,4 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,08/05/2025,18/05/2025,Tin thường,42374969
3,"Đường Sư Vạn Hạnh, Phường 12, Quận 10, Hồ Chí ...",Nhà riêng,109 m²,"23,8 tỷ",5 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,15/05/2025,25/05/2025,Tin thường,42342057
4,"Đường Bạch Đằng, Phường 24, Bình Thạnh, Hồ Chí...",Nhà mặt phố,52 m²,"8,9 tỷ",NaN,NaN,3 tầng,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,04/05/2025,14/05/2025,Tin thường,42571965


### **2.2 Tổng quát**:
#### **2.2.1 Số dòng và số cột:**

In [3]:
num_rows, num_cols = real_estate_for_sale_df.shape

print(f'Num of rows:  {num_rows}')
print (f'Num of columns:  {num_cols}')

Num of rows:  64078
Num of columns:  14


#### **2.2.2 Ý nghĩa của từng cột?**

- The data we used here is collected by crawling raw data from the website https://batdongsan.com.vn/        
- Each line is a record of a advertisement of real estate for sale. So there isn't any line that has different meaning.

#### **2.2.3 Số cột bị trùng lặp:**

In [4]:
duplicate = real_estate_for_sale_df.duplicated().sum()

print (f' Nums of duplicated rows: {duplicate}')

 Nums of duplicated rows: 28366


We can see that there are duplicated rows in the dataset. The reasons here are some advertisements could be reposted in the website, or some errors during the data-collecting process may lead to the duplicated data.      
So, we will drop these duplicated rows, then reset the index of the dataframe.

In [5]:
real_estate_for_sale_df.drop_duplicates(inplace= True)
real_estate_for_sale_df = real_estate_for_sale_df.reset_index(drop=True)

#### **2.2.4 Tỉ lệ dữ liệu thiếu của từng cột:**


In [6]:
# Finf the missing value
def missing_ratio(column):
    missing_values = column.isnull().sum()
    total_values = len(column)
    return (missing_values / total_values) * 100

missing_ratios_df = real_estate_for_sale_df.agg(missing_ratio).to_frame()
missing_ratios_df.columns = ['Missing ratio']

missing_ratios_df

,Missing ratio
Address,0.000000
Type,0.000000
Area,0.000000
Price,0.000000
Bedroom,33.448141
Toilet,100.000000
Floor,46.463374
Furniture,51.512097
Direction,75.383625
Legal,15.249776


- Chúng ta có thể thấy các trường Address, Type, Area, Price, Posting date, Ad type, Ad code  không có giá trị bị thiếu. Điều này dễ hiểu vì: Địa chỉ, Loại hình và Giá là các dữ liệu cơ bản mà một bài đăng phải có

- Bên cạnh đó, các trường như Bedroom, Floor, Furniture có tính chất đặc thù hơn. Bộ dữ liệu bao gồm nhiều loại bất động sản, bao gồm các danh mục như Đất nền dự án, Đất bán, Trang trại, khu nghỉ dưỡng, những loại bất động sản này có thể không có thông tin về các trường trên. Vì vậy, việc các trường này có giá trị bị thiếu là hợp lý. Chúng ta sẽ xem xét rõ hơn ở phần tiếp theo.

- Các trường còn lại có tỷ lệ giá trị bị thiếu lớn như Toilet và Direction. Do đó, việc tiền xử lý dữ liệu là rất cần thiết trước khi tiến hành phân tích.

### **2.3 Ý nghĩa của từng cột**


In [7]:
real_estate_for_sale_df.columns.to_list()

['Address',
 'Type',
 'Area',
 'Price',
 'Bedroom',
 'Toilet',
 'Floor',
 'Furniture',
 'Direction',
 'Legal',
 'Posting date',
 'Expiry date',
 'Ad type',
 'Ad code']

- Địa chỉ: Địa chỉ của bất động sản. Với bất động sản là căn hộ, địa chỉ cũng có thể bao gồm tên dự án của bất động sản.
- Loại hình: Danh mục của bất động sản.
- Diện tích: Diện tích của bất động sản.
- Giá: Giá của bất động sản. Có thể là giá tổng hoặc chỉ là giá trên mỗi m².
- Phòng ngủ: Số lượng phòng ngủ trong bất động sản.
- Nhà vệ sinh: Số lượng nhà vệ sinh trong bất động sản.
- Tầng: Số lượng tầng.
- Nội thất: Tình trạng nội thất của bất động sản.
- Hướng: Hướng của bất động sản.
- Pháp lý: Một số thông tin pháp lý của bất động sản.
- Ngày đăng: Ngày bài quảng cáo được đăng.
- Ngày hết hạn: Ngày bất động sản được kích hoạt.
- Loại quảng cáo: Loại hình quảng cáo.
- Mã quảng cáo: Mã của bài quảng cáo.

#### **2.3.1 Kiểu dữ liệu:**


In [8]:
cols_type = real_estate_for_sale_df.dtypes
cols_type

Address          object
Type             object
Area             object
Price            object
Bedroom          object
Toilet          float64
Floor            object
Furniture        object
Direction        object
Legal            object
Posting date     object
Expiry date      object
Ad type          object
Ad code           int64
dtype: object

- Nearly all of the columns is in Object type. These columns will not suitable for the further analysis. So we need to do some preprocessing on the data types.
- `Area`, `Price`, `Bedroom`, `Toilet` and `Floor` should be numerical columns, so we will convert them into numeric data types.
- `Posting date` should be in `datetime` type
- `Expiry date`, `Ad type`, `Ad code` is not so meaningful for our analysis, so we should remove it later.


### **2.4 Tiền xử lí dữ liệu:**

#### **2.4.1.Diện tích :**


In [9]:
real_estate_for_sale_df['Area'].to_frame()

,Area
0,66 m²
1,55 m²
2,45 m²
3,109 m²
4,52 m²
...,...
35707,100 m²
35708,74 m²
35709,93 m²
35710,56 m²


Thống nhất về một kiểu đơn vị

In [10]:
area_values = real_estate_for_sale_df[real_estate_for_sale_df['Area'].notna()]['Area'].to_list()
unit_list = []

# Extract the unit of each area.
for area in area_values:
    unit_list.append(area.split(' ')[1])

set(unit_list)

{'m²'}

Thống nhất về một kiểu format

In [11]:

def clean_area(area):
    
    if pd.isna(area):
        return area
    
    area = area.split(' ')[0]
    # Handle 1,233.4 case, convert it into 1233.4
    if (',' in area) and ('.' in area):
        cleaned_area = area.replace('.', '').replace(',', '.')

    # Case 334,4 -> 334.4
    elif ',' in area:
        
        cleaned_area = area.replace(',', '.')
    else:
        # print (area)
        cleaned_area = area
    
    # Case 1.344 -> 1344
    if '.' in area:
        count = len(area.split('.')[-1])
        if count == 3:
            cleaned_area = area.replace('.', '')
    
    return cleaned_area

real_estate_for_sale_df.loc[:,'Area'] = real_estate_for_sale_df.loc[:,'Area'].apply(clean_area)
real_estate_for_sale_df['Area'] = real_estate_for_sale_df['Area'].astype('float64')

# Drop the area that is nan
real_estate_for_sale_df = real_estate_for_sale_df[real_estate_for_sale_df['Area'].notna()]
real_estate_for_sale_df.rename(columns={'Area': 'Area(m2)'}, inplace=True)

#### **2.4.2 Giá tiền:**
Thống nhất về một kiểu đơn vị

In [12]:

price_values = real_estate_for_sale_df['Price'].to_list()

unit_list = []
value_list = []
for price in price_values:
    unit_list.append(price.split(' ')[1])
    value_list.append(price.split(' ')[0])

set(unit_list)

{'nghìn/m²', 'thuận', 'triệu', 'triệu/m²', 'tỷ', 'tỷ/m²'}

Đơn vị chính sẽ là VND. Đối với những dữ liệu là "Thỏa thuận" thì sẽ được chuyển đổi thành NaN.

In [13]:
# raise ExceptionGroup("NotImplemented")
def update_price(price, area):
    new_price = 0

    if 'nghìn/m²' in price:
        new_price = float(price.split(' ')[0]) * area

    elif 'triệu' in price:
        new_price, unit = price.split(' ')
        if unit.strip() == 'triệu':
            new_price = float(new_price.replace(',', '.'))
            new_price *= 1000000
        elif unit.strip() == 'triệu/m²':
            new_price = float(new_price.replace(',', '.'))
            new_price *= area * 1000000

    elif 'tỷ' in price:
        new_price, unit = price.split(' ')
        if unit.strip() == 'tỷ/m²':
            new_price = float(new_price.replace(',', '.'))
            new_price *= area * 1000000000
        else:
            new_price = float(new_price.replace(',', '.'))
            new_price *= 1000000000
   
    return "{:0,.2f}".format(new_price) if new_price != 0 else np.nan


real_estate_for_sale_df['Price'] = real_estate_for_sale_df.apply(lambda row: update_price(row['Price'], row['Area(m2)']), axis=1)

real_estate_for_sale_df.rename(columns={'Price': 'Price(VNĐ)'}, inplace=True)


#### **2.4.3 Bedroom, Toilet, Floor:**
Xử lí dữ liệu 3 cột này.

In [14]:

column_list = ['Bedroom', 'Toilet', 'Floor']
unit_dict = {}

# Extract the value and unit into list, then store them into a dictionary
for column in column_list:
    column_mask = real_estate_for_sale_df[column].notna()
    column_values = real_estate_for_sale_df.loc[column_mask, column]
    
    unit_list = []
    value_list = []
    
    for value in column_values:
        unit_list.append(value.split(' ')[1])
        value_list.append(value.split(' ')[0])
    
    unit_dict[column] = (set(unit_list), column_mask, value_list)

for key, value in unit_dict.items():
    print(key, value[0])


Bedroom {'phòng'}
Toilet set()
Floor {'tầng'}


In [15]:
# Set the new value and convert the data type into float
for key, value in unit_dict.items():
    real_estate_for_sale_df.loc[value[1], key] = value[2]
    real_estate_for_sale_df[key] = real_estate_for_sale_df[key].astype('float64')


In [16]:
real_estate_for_sale_df.rename(columns={'Bedroom': 'Bedroom(Phòng)'}, inplace=True)
real_estate_for_sale_df.rename(columns={'Toilet': 'Toilet(Phòng)'}, inplace=True)
real_estate_for_sale_df.rename(columns={'Floor': 'Floor(Tầng)'}, inplace=True)



### **2.5 Tiền xử lí dữ cho các cột dữ liệu kiểu categorical :**
#### **2.5.1 Addresss:**


##### **Trích xuất dữ liệu để lấy Quận, Huyện:**

In [17]:
def extract_district(address):
    # Kiểm tra nếu address không hợp lệ (NaN, rỗng, hoặc không phải chuỗi)
    if not isinstance(address, str) or not address.strip():
        return 'Không'
    
    # Biểu thức chính quy để tìm quận/huyện/thành phố
    pattern = r'(?:Quận|Huyện|Thành phố|Q\.|TP\.)\s*([\w\s]+?)(?:,|$|\s*Hồ Chí Minh|\s*Hà Nội)'
    match = re.search(pattern, address, re.IGNORECASE)
    
    if match:
        return match.group(1).strip()
    return 'Không'

Tạo một cột để lưu dữ liệu "District"

In [18]:

real_estate_for_sale_df.loc[:,'District'] = real_estate_for_sale_df['Address'].apply(extract_district)

real_estate_for_sale_df = real_estate_for_sale_df[real_estate_for_sale_df['District'] != 'Không']

real_estate_for_sale_df['District'].value_counts()

District
9                  3498
2                  3403
7                  2967
1                  1792
12                 1570
3                  1119
10                 1075
8                   927
6                   518
5                   484
4                   434
11                  327
Thủ Đức              12
Tân Phú              12
Cần Giờ              11
Bình Tân             11
Thanh Quan           10
Nhà Bè                7
Bình Thạnh            7
Bình Chánh            5
Gò Vấp                4
Hồ Chí Minh           2
Tân Bình              2
Phú Nhuận             1
Củ Chi                1
Phú nhuận             1
THủ Đức               1
Thủ  Đức              1
1 HẺM VIP 4x23m       1
                      1
Name: count, dtype: int64

##### **Trích xuất dữ liệu để lấy dữ liệu Dự Án:**


In [19]:
def extract_project(address):
    if 'Dự án' in address:
        return address.split(',')[0]
    else:
        return 'Không'

In [20]:
real_estate_for_sale_df.loc[:, 'Project'] = real_estate_for_sale_df['Address'].apply(extract_project)

#### **2.5.2 Furniture:**

- Phân loại dữ liệu cho cột Furniture

In [21]:
# Just define some key word for each level of furniture
deluxe_furniture_keywords = ["cao cấp", "ntcc", "sang", "5 sao", "nhập khẩu", "hiện đại", "đạt chuẩn"]
fully_furniture_keywords = ["đủ", "full", "tặng", "toàn bộ", "hoàn chỉnh", "hoàn thiện", "đẹp", "đã làm", "mới"]
basic_furniture_keywords = ["cơ bản", "ít", "ntcb", "có", "ok", "dính tường", "căn bản"]
none_furniture_keywords = ["thô", "trống", "không", "ko"]

# Funtion to normalize the data.
def normalize_text(text):
    if pd.isnull(text):
        return "Không có thông tin"
    # Normalize to NFC form
    normalized_text = unicodedata.normalize('NFC', text.lower())
    return normalized_text


def clean_furniture(furniture):
    if pd.isnull(furniture):
        return "Không có thông tin"
  
    normalized_furniture = unicodedata.normalize('NFC', furniture.lower())

    # Find and set the funiture status based on the keywords
    if any(keyword in normalized_furniture for keyword in deluxe_furniture_keywords):
        return "Nội thất sang trọng"
    elif any(keyword in normalized_furniture for keyword in fully_furniture_keywords) or len(normalized_furniture.split(',')) >= 4:
        return "Nội thất đầy đủ"
    elif any(keyword in normalized_furniture for keyword in basic_furniture_keywords) or 2 <= len(normalized_furniture.split(',')) < 4:
        return "Nội thất cơ bản"
    elif any(keyword in normalized_furniture for keyword in none_furniture_keywords):
        return "Không nội thất"
    else:
        return "Khác"

real_estate_for_sale_df['Furniture'] = real_estate_for_sale_df['Furniture'].apply(clean_furniture)


In [22]:
real_estate_for_sale_df['Furniture'].value_counts()

Furniture
Không có thông tin     9255
Nội thất đầy đủ        5370
Nội thất cơ bản        2626
Không nội thất          632
Nội thất sang trọng     280
Khác                     41
Name: count, dtype: int64

#### **2.5.3 Legal:**

- Phân loại kiểu dữ liệu cho cột "legal"

In [23]:

full_legal_keywords = [ "sổ đỏ/ sổ hồng","có sổ", "đầy đủ", "sang", "chuẩn", "rõ ràng", "ok"]
only_land_keywords = ["sổ đỏ"]
only_house_keywords = ["sổ hồng", "shr"]
contract_keywords = ["hợp đồng", "hđmb"]
waiting_keywords = ["chờ"]


def clean_legal(legal):
    if pd.isnull(legal):
        return "Không có thông tin"

    normalized_legal = unicodedata.normalize('NFC', legal.lower())
    
    # Set legality status based on the keywords
    legal = normalized_legal.lower().strip()
    if any(keyword in legal for keyword in full_legal_keywords):
        return 'Sổ đỏ và sổ hồng'
    elif any(keyword in legal for keyword in only_land_keywords):
        return 'Sổ đỏ'
    elif any(keyword in legal for keyword in only_house_keywords):
        return'Sổ hồng'
    elif any(keyword in legal for keyword in contract_keywords):
        return 'Hợp đồng mua bán'
    elif any(keyword in legal for keyword in waiting_keywords):
        return 'Đang chờ sổ'
    else:
        
        return 'Khác'

In [24]:
real_estate_for_sale_df['Legal'] = real_estate_for_sale_df['Legal'].apply(clean_legal)
real_estate_for_sale_df['Legal'].value_counts()


Legal
Sổ đỏ và sổ hồng      12179
Không có thông tin     3234
Hợp đồng mua bán       1849
Sổ hồng                 557
Đang chờ sổ             214
Khác                     99
Sổ đỏ                    72
Name: count, dtype: int64

#### **2.5.4 Posting date:**
Chuyển kiểu dữ liệu thành datetime

In [25]:
real_estate_for_sale_df['Posting date'] = pd.to_datetime(real_estate_for_sale_df['Posting date'], format = '%d/%m/%Y')

### **2.7 Xóa các cột dữ liệu không cần thiết**:


In [26]:
real_estate_for_sale_df['Type'].unique()

array(['Nhà riêng', 'Căn hộ chung cư', 'Nhà biệt thự, liền kề', 'Đất bán',
       'Nhà mặt phố', 'Shophouse nhà phố thương mại', 'Kho, nhà xưởng',
       'Đất nền dự án', 'Condotel', 'Bất động sản khác'], dtype=object)

In [27]:
deleted_types = ['Kho, nhà xưởng', 'Shophouse nhà phố thương mại', 'Trang trại, khu nghỉ dưỡng', 'Bất động sản khác']

real_estate_for_sale_df = real_estate_for_sale_df[~real_estate_for_sale_df['Type'].isin(deleted_types)]
real_estate_for_sale_df.drop(columns=['Direction', 'Expiry date', 'Ad type', 'Ad code', 'Address'], inplace= True)

### **2.8 Kiểm tra tính hợp lí của dữ liệu**


#### **2.8.1 Kiểm tra outliner trong dữ liệuliệu**



In [28]:
real_estate_for_sale_df.loc[:,'Price(VNĐ)'] = real_estate_for_sale_df.loc[:,'Price(VNĐ)'].str.replace(',', '')
real_estate_for_sale_df.loc[:,'Price(VNĐ)'] = pd.to_numeric(real_estate_for_sale_df['Price(VNĐ)'], errors='coerce')
real_estate_for_sale_df['Price(VNĐ)'] = real_estate_for_sale_df['Price(VNĐ)'].astype(float)

In [29]:
columns_to_plot = ['Area(m2)', 'Price(VNĐ)', 'Bedroom(Phòng)', 'Floor(Tầng)']
grouped_by_type_df = real_estate_for_sale_df.groupby('Type')
fig, axes = plt.subplots(nrows=6, ncols=5, figsize=(70, 150))

index = 0
for key, data in grouped_by_type_df:
    for column in columns_to_plot:

        ax = axes[index // 5, index % 5]
        
        sns.boxplot(x = data['Type'], y = data[column].dropna(), ax = ax)
        ax.set_title(f"Outliers for {column} - {key}", fontsize = 16, fontweight = 'bold', pad = 10)
        ax.set_xlabel("Type", fontsize = 12, fontweight = 'bold')
        ax.set_ylabel(column, fontsize = 12, fontweight = 'bold')
        index += 1

fig = plt.gcf()
fig.set_facecolor('#ffe6cc')
plt.show()

Thông qua các biểu đồ hộp (box plot), chúng ta thấy rằng có rất nhiều giá trị ngoại lai (outliers) xuất hiện trong tập dữ liệu của chúng ta. Những giá trị ngoại lai này có thể gây ảnh hưởng tiêu cực đến phân tích trong tương lai, đồng thời mô hình của chúng ta sẽ không đạt được hiệu suất cao. Vì vậy, theo cách đơn giản nhất, chúng ta chỉ cần loại bỏ tất cả các giá trị ngoại lai.

Đầu tiên, hãy xem xét các loại Đất bán và Đất nền dự án, thông thường, những loại này sẽ không có giá trị trong các cột Bedroom, Toilet và Floor, vì vậy chỉ giữ lại tất cả các mẫu thuộc các loại này mà không có giá trị trong các cột đó.

In [30]:

filtered_df = real_estate_for_sale_df[real_estate_for_sale_df['Type'].isin(['Đất bán', 'Đất nền dự án']) &
                                        ((real_estate_for_sale_df['Bedroom(Phòng)'] > 0) |
                                        (real_estate_for_sale_df['Toilet(Phòng)'] > 0) |
                                        (real_estate_for_sale_df['Floor(Tầng)'] > 0))]

real_estate_for_sale_df.drop(filtered_df.index, inplace=True)

Được rồi, giờ chúng ta đã loại bỏ tất cả các giá trị ngoại lai cho loại Đất bán và Đất nền dự án, tiếp theo, chúng ta sẽ chuyển sang các cột phức tạp hơn.

Với các loại bất động sản khác, chúng ta có thể thấy rằng có những giá trị xa hơn nhiều so với giá trị trung vị. Vì vậy, chúng ta sẽ triển khai cách phát hiện và loại bỏ các giá trị ngoại lai cho từng loại bất động sản.

In [31]:
columns_to_detect = ['Area(m2)', 'Price(VNĐ)', 'Bedroom(Phòng)', 'Floor(Tầng)']

cleaned_real_estate_df = real_estate_for_sale_df.copy()

for column in columns_to_detect:
    column_values = cleaned_real_estate_df[column].dropna().to_numpy()

    q1 = np.percentile(column_values, 25)
    q3 = np.percentile(column_values, 75)
    iqr = q3 - q1

    lower_threshold = q1 - 1.5 * iqr
    upper_threshold = q3 + 1.5 * iqr

    outliers_mask = (cleaned_real_estate_df[column] < lower_threshold) | (cleaned_real_estate_df[column] > upper_threshold)

    cleaned_real_estate_df = cleaned_real_estate_df[~outliers_mask]


### **2.6 Xử lí dữ liệu bị thiếu**


First, check the missing ratio now

In [32]:

missing_ratios_df = cleaned_real_estate_df.agg(missing_ratio).to_frame()
missing_ratios_df

,0
Type,0.000000
Area(m2),0.000000
Price(VNĐ),11.416534
Bedroom(Phòng),26.852683
Toilet(Phòng),100.000000
Floor(Tầng),55.349126
Furniture,0.000000
Legal,0.000000
Posting date,0.000000
District,0.000000


 Các cột Bedroom, Toilet, Floor không phải là thuộc tính của một số loại bất động sản. Vì vậy, đối với Đất nền dự án, Đất bán, Trang trại, khu nghỉ dưỡng, các giá trị NaN sẽ được thay thế bằng -1. Đối với các loại bất động sản khác, tôi cũng sẽ nhóm theo danh mục và điền giá trị trung bình của từng danh mục.

In [33]:
replace_nan_types = ['Đất nền dự án', 'Đất bán', 'Trang trại, khu nghỉ dưỡng']
columns = ['Bedroom(Phòng)', 'Floor(Tầng)']

for col in columns:
    mask = cleaned_real_estate_df['Type'].isin(replace_nan_types)
    cleaned_real_estate_df.loc[mask, col] = cleaned_real_estate_df.loc[mask, col].fillna(-1)
    cleaned_real_estate_df.loc[~mask , col] = cleaned_real_estate_df.loc[ ~mask, col].fillna(round(cleaned_real_estate_df.loc[~mask, col].mean(),0))

In [34]:
missing_ratios_df = cleaned_real_estate_df.agg(missing_ratio).to_frame()
missing_ratios_df

,0
Type,0.000000
Area(m2),0.000000
Price(VNĐ),11.416534
Bedroom(Phòng),0.000000
Toilet(Phòng),100.000000
Floor(Tầng),0.000000
Furniture,0.000000
Legal,0.000000
Posting date,0.000000
District,0.000000


In [35]:
cleaned_real_estate_df = cleaned_real_estate_df.drop(columns=['Toilet(Phòng)'])

In [36]:
missing_ratios_df = cleaned_real_estate_df.agg(missing_ratio).to_frame()
missing_ratios_df

,0
Type,0.000000
Area(m2),0.000000
Price(VNĐ),11.416534
Bedroom(Phòng),0.000000
Floor(Tầng),0.000000
Furniture,0.000000
Legal,0.000000
Posting date,0.000000
District,0.000000
Project,0.000000


In [38]:
cleaned_real_estate_df.head()

,Type,Area(m2),Price(VNĐ),Bedroom(Phòng),Floor(Tầng),Furniture,Legal,Posting date,District,Project
0,Nhà riêng,66.0,NaN,4.0,3.0,Không có thông tin,Sổ đỏ và sổ hồng,2025-05-03,9,Không
1,Căn hộ chung cư,55.0,2.700000e+09,1.0,3.0,Nội thất đầy đủ,Sổ đỏ và sổ hồng,2025-05-03,9,Dự án Fuji Residence
3,Nhà riêng,109.0,2.380000e+10,5.0,3.0,Không có thông tin,Sổ đỏ và sổ hồng,2025-05-15,10,Không
8,Nhà riêng,92.5,9.100000e+09,4.0,3.0,Nội thất đầy đủ,Sổ đỏ và sổ hồng,2025-05-13,7,Không
9,Nhà riêng,80.0,7.500000e+09,4.0,4.0,Nội thất đầy đủ,Sổ đỏ và sổ hồng,2025-05-14,7,Không


### Lưu dữ liệu đã được tiền xử lí

In [39]:
cleaned_real_estate_df.to_csv("../Data/cleaned_real_estate_for_sale.csv", index= False)